# Filters

We can use filters from various reasons.
* We would like to reduce the noise on the image.
* We prepare the image for further processing steps.
* The result of the filter is just interesting for us.

$\rhd$ Check the filters in GIMP!

## Noises

There are many kind of noises, for instance:

* Dot like noise, salt-and-pepper noise,
* White noise,
* Blurry parts,
* Colorspace problems (shifting),
* Over exponation,
* Perspective torsion, torsion of lenses,
* Motion blur,
* JPG noise,
* Overlapped, missing image parts.

We can find more noise types and reasons here:

* https://en.wikipedia.org/wiki/Image_noise

$\rhd$ Show an example for the JPG noise (for instance by using GIMP)!

In [ ]:
import cv2

In [ ]:
image_path = 'samples/photos/dragon.jpg'
image = cv2.imread(image_path)

Distance

In [ ]:
import numpy as np

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
image = cv2.resize(image, (86, 115))

In [ ]:
image.shape

In [ ]:
result = image.copy()
for i in range(115):
    for j in range(86):
        r, g, b = image[i, j, :].astype(int)
        # value = (r + g + b) / 3
        value = np.sqrt(r*r + g*g + b*b) / np.sqrt(3)
        value = int(value)
        # print(r, g, b, value)
        result[i, j, :] = [value, value, value]

In [ ]:
plt.imshow(result)

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(image, cmap='gray')

Salt-and-pepper noise

In [ ]:
n_rows, n_columns = image.shape

In [ ]:
import random

In [ ]:
salted_image = image.copy()
for _ in range(200):
    i = random.randrange(n_rows)
    j = random.randrange(n_columns)
    if image[i, j] < 128:
        salted_image[i, j] = 255
    else:
        salted_image[i, j] = 0

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(salted_image, cmap='gray')
plt.show()
plt.close()

White noise

In [ ]:
plt.imshow(image, cmap='gray')

In [ ]:
import numpy as np

In [ ]:
noisy_image = image.copy()
for i in range(n_rows):
    for j in range(n_columns):
        noise = np.random.normal(0, 15)
        value = image[i, j] + noise
        if value < 0:
            value = 0
        if value > 255:
            value = 255
        noisy_image[i, j] = value

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(noisy_image, cmap='gray')
plt.show()
plt.close()

$\rhd$ Check the histograms of the original and noised images!

## Noise reduction

It is an estimation problem.
* We consider the available information.
* We try to guess the missing ones.

NOTE: Without a well-known model of the image, it is impossible (theoretically) to reconstruct the original image.

## Convolution

* https://en.wikipedia.org/wiki/Convolution

Let consider a function $f: \mathbb{R} \rightarrow \mathbb{R}$ as the signal which should be convoluted.

Let function $g: \mathbb{R} \rightarrow \mathbb{R}$ a convolution function (*kernel*).

The convolution can be denoted and defined as
$$
(f * g)(t) = \int_{-\infty}^{+\infty} \! f(\tau) g(t - \tau) \; \mathrm{d}\tau.
$$

In image processing usually it is enough to use the discrete, definite case.

$\rhd$ Show an example of one-dimensional discrete convolution!

The kernel is a matrix.
* We slide a window over the image (*sliding window*).
* The resolution of the kernel is odd-by-odd sized.
* The kernel is a squared matrix.

$$
g \in \mathbb{R}^{(2k + 1) \times (2k + 1)}
$$

The linear convolution (as a simple, special case) can be defined as:
$$
(f * g)(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k}
f(x + j, y + i) \cdot g(j, i).
$$

## Average filters

* The resulted intensity is a moving average.
* https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html

$$
g = \dfrac{1}{9}
\begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1 \\
\end{bmatrix}
$$

In [ ]:
kernel = np.ones((3, 3), np.float32) / 9

In [ ]:
kernel = np.ones((5, 5), np.float32) / 25

In [ ]:
avg_image = cv2.filter2D(image, -1, kernel)

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(avg_image, cmap='gray')
plt.show()
plt.close()

$\rhd$ Check the effect of consecutive average filtering! (For instance by saving the sequence of images to files.)

## Gaussian filters

* The weights in the kernel approximates the two-dimensional Gaussian distribution.

$$
g = \dfrac{1}{16}
\begin{bmatrix}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1 \\
\end{bmatrix}
$$

$$
g = \dfrac{1}{273}
\begin{bmatrix}
1 & 4 & 7 & 4 & 1 \\
4 & 16 & 26 & 16 & 4 \\
7 & 26 & 41 & 26 & 7 \\
4 & 16 & 26 & 16 & 4 \\
1 & 4 & 7 & 4 & 1 \\
\end{bmatrix}
$$

In [ ]:
kernel = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
]) / 16

In [ ]:
gauss_image = cv2.filter2D(image, -1, kernel)

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(gauss_image, cmap='gray')
plt.show()
plt.close()

In [ ]:
blurred_image = cv2.GaussianBlur(image, (9, 9), 0)

In [ ]:
blurred_image = cv2.GaussianBlur(image, (15, 15), 0)

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(blurred_image, cmap='gray')
plt.show()
plt.close()

$\rhd$ Check various parametrizations of the kernel!

$\rhd$ Compare the result of the average and gauss filters (for instance by the difference)!

$\rhd$ Try to check the values of the matrices!

$\rhd$ Write a function, which can provide larger weight matrices!

In [ ]:
# Above Python 3.8
# from statistics import NormalDist
# NormalDist(mu=0, sigma=1).cdf(1.96)

In [ ]:
from scipy.stats import norm

In [ ]:
xs = np.linspace(-5, 5, 100)

In [ ]:
ys = norm.cdf(xs)

In [ ]:
plt.figure()
plt.plot(xs, ys)
plt.show()
plt.close()

## Median filters

* It replaces the intensity by the median of the intensities of the sliding window.
* Against the linear filters, it does not add new intensity to the image.
* It is an ideal filter for reducing salt-and-pepper noise.
* It requires more calculation than the linear convolutional filters.

In [ ]:
# median_image = cv2.medianBlur(image, 3)
median_image = cv2.medianBlur(salted_image, 3)

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(salted_image, cmap='gray')
plt.show()
plt.close()

plt.figure(figsize=(5, 8))
plt.imshow(median_image, cmap='gray')
plt.show()
plt.close()

$\rhd$ Check the effect of consecutive median filtering! (For instance by saving the sequence of images to files.)

## Bilateral filtering

https://people.csail.mit.edu/sparis/bf_course/

## Estimation of the effectiveness

In [ ]:
diff = image - blurred_image

In [ ]:
plt.figure(figsize=(5, 8))
plt.imshow(median_image, cmap='coolwarm')
plt.show()
plt.close()

Norms

* https://en.wikipedia.org/wiki/Matrix_norm
* https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html

$\rhd$ Measure the effectiveness of gaussian and median filtering in the case of some synthetic noises!